In [1]:
import caffe
import cv2
import numpy as np

# Load caffe model

In [2]:
class Predictor:
    
    def preprocess(self, resized):
        ret = resized.astype('f4')
        ret -= self.mean
        ret /= (1.e-6+ self.std)
        return  ret
    
    def predict(self, resized):
        """
        @resized: image 40,40 already pre processed 
        """         
        self.net.blobs['data'].data[...] = cv2.split(resized)
        prediction = self.net.forward()['Dense2'][0]
        return prediction
        
    def __init__(self, protoTXTPath, weightsPath):
        caffe.set_mode_cpu()
        self.net = caffe.Net(protoTXTPath,weightsPath,caffe.TEST)
        self.mean = cv2.imread('./trainMean.png').astype('float')
        self.std  = cv2.imread('./trainSTD.png').astype('float')

In [3]:
predictor = Predictor("./vanilla_deploy.prototxt","./vanillaCNN.caffemodel")       #load caffe model and weights

a=cv2.imread("./test.png")
a=cv2.resize(a,(40,40))                     #load test image

img=predictor.preprocess(a)           #pre processing

points=predictor.predict(img)       #predict facial landmark

print(points)


[-0.22493494 -0.24115832  0.18771622 -0.25451189  0.02025338 -0.02075372
 -0.23037848  0.14349605  0.20071335  0.13110712]


##  Draw landmarks in image

In [4]:

landmark=(points+0.5)*128
print(landmark)

a=cv2.resize(a,(128,128))

cv2.circle(a,(landmark[0],landmark[1]),2,(0,255,0))
cv2.circle(a,(landmark[2],landmark[3]),2,(0,255,0))
cv2.circle(a,(landmark[4],landmark[5]),2,(0,255,0))
cv2.circle(a,(landmark[6],landmark[7]),2,(0,255,0))
cv2.circle(a,(landmark[8],landmark[9]),2,(0,255,0))

# cv2.imshow("1",a)
# cv2.waitKey(0)

[ 35.20832825  33.13173676  88.02767944  31.42247772  66.59243011
  61.34352493  34.51155472  82.36749268  89.69130707  80.78170776]


# Load MXNet model

In [5]:
import mxnet as mx

model=mx.model.FeedForward.load('vanilla',1,num_batch_size=1)     #load model

In [6]:
b=np.zeros((1,3,40,40),dtype=np.float32)      #change chanels into shape[1,3,40,40]
b[0,0,:,:]=img[:,:,2]
b[0,1,:,:]=img[:,:,1]
b[0,2,:,:]=img[:,:,0]                               #Swapping BGR of caffe into RGB in mxnet

In [7]:
points=model.predict(b)
print(points)

landmark=(points+0.5)*128       #print landmarks
print(landmark[0])


[[-0.22493497 -0.2411584   0.18771638 -0.25451207  0.0202535  -0.02075362
  -0.23037851  0.14349589  0.2007134   0.13110715]]
[ 35.20832443  33.13172531  88.0276947   31.42245483  66.59244537
  61.34353638  34.5115509   82.36747742  89.6913147   80.78171539]


In [8]:
a=cv2.imread("./test.png")

landmark=landmark[0]
cv2.circle(a,(landmark[0],landmark[1]),2,(0,255,0))
cv2.circle(a,(landmark[2],landmark[3]),2,(0,255,0))
cv2.circle(a,(landmark[4],landmark[5]),2,(0,255,0))
cv2.circle(a,(landmark[6],landmark[7]),2,(0,255,0))
cv2.circle(a,(landmark[8],landmark[9]),2,(0,255,0))

cv2.imwrite("landmark.jpg",a)   #save landmarks

True